<a href="https://colab.research.google.com/github/angelaaaateng/DEPA_Project/blob/master/Python_GCP_Master_Final0606.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [18]:
#Install packages

!pip install gcsfs
!pip install praw
!pip install psaw

#Import packages

import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import praw
from psaw import PushshiftAPI
import gcsfs
df_names = pd.read_csv('gs://cryptodb-files/crypto_names.csv')


FileNotFoundError: ignored

In [0]:
df_names.head()

,coin_id,name,symbol,slug
0,0,0x,ZRX,0x
1,1,Aeternity,AE,aeternity
2,2,Aion,AION,aion
3,3,Ardor,ARDR,ardor
4,4,Ark,ARK,ark


In [0]:

# # Reddit Table
# #### Reddit posts from /r/Cryptocurrency from 2017-2019


#Connect to reddit API
reddit = praw.Reddit(client_id='GjfUHQE8AYnXLg', client_secret='PfbhtsXJGAAUNiEyHPRGPuFJ0ro', user_agent='DEPA_Project')
api = PushshiftAPI()

#Extract all posts to /r/CryptoCurrency from 2017-2019

start_epoch=int(dt.datetime(2015, 1, 1).timestamp())
end_epoch = int(dt.datetime(2019, 6, 1).timestamp())

reddit_data = pd.DataFrame(api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit='CryptoCurrency',
                            filter=['author', 'title', 'subreddit', 'num_comments', 'created', 'score']))

#Convert the created_utc column to a date 
reddit_data['created_utc']=(pd.to_datetime(reddit_data['created_utc'],unit='s'))

#remove redundant columns
reddit_data = reddit_data.drop(['created', 'd_'], axis = 1)


#Rename created_utc to date
reddit_data.rename(columns = {'created_utc':'date'}, inplace = True)


#Drop date column
#reddit_data2 = reddit_data2.drop('date', axis = 1)


reddit_data.index.names = ['post_id']

reddit_data.head()


,author,date,num_comments,score,subreddit,title
post_id,,,,,,
0,Mahaprajapati,2019-05-31 23:56:44,0,1,CryptoCurrency,Swap (XWP) added to CNPool.cc today
1,webnowcompany,2019-05-31 23:55:15,0,1,CryptoCurrency,Millionaire Alexander Amado Johnson and his wi...
2,sebikun,2019-05-31 23:48:29,4,29,CryptoCurrency,"How is OmiseGOing? By Kasima Tharnpipitchai, D..."
3,albuquerquetulio,2019-05-31 23:47:14,0,1,CryptoCurrency,Gemini is down??
4,cryptotradinglife,2019-05-31 23:46:36,0,1,CryptoCurrency,Analyst: Regardless of Newest Bitcoin Pullback...


In [0]:
# # Create reddit_coins Join Table 
# #### Matches the reddit posts in reddit_data table to the coins in coins table


#Zip the names into tuple of (name, slug, symbol)
zipped_names = list(zip(df_names['name'],df_names['slug'],df_names['symbol']))

#Create a search list - seperating the three terms with OR operator (|)
search_list = []
for (name, slug, symbol) in zipped_names:
    listt = [name, slug, symbol]
    pat = '|'.join(listt)
    search_list.append(pat)


#Add boolean columns
import re
dummy_df = pd.DataFrame(dict((name, reddit_data.title.str.contains(name, re.IGNORECASE))
                             for name in search_list))



#Convert dummy columns into rows with post_id as the index
i, j = np.where(dummy_df)

coins_mentioned_series = pd.Series(dict(zip(zip(i, j), dummy_df.columns[j])))



#Create final join table between reddit and coins table. Rename columns of dataframe. 
coin_reddit_join = pd.DataFrame(coins_mentioned_series).reset_index()
coin_reddit_join.columns = ['post_id', 'coin_id', 'coin_name']
coin_reddit_join = coin_reddit_join.drop('coin_name', axis = 1)


coin_reddit_join2 = coin_reddit_join.set_index(['post_id', 'coin_id'],)


coin_reddit_join2


Empty DataFrame
Columns: []
Index: [(1, 77), (2, 66), (4, 11), (4, 77), (5, 72), (7, 11), (7, 95), (8, 77), (11, 77), (16, 4), (16, 5), (16, 11), (16, 35), (16, 36), (16, 66), (16, 77), (17, 4), (17, 11), (18, 77), (22, 77), (25, 77), (26, 77), (28, 4), (28, 77), (29, 11), (29, 12), (34, 11), (35, 77), (36, 32), (36, 85), (37, 11), (37, 12), (37, 16), (37, 35), (37, 52), (39, 11), (41, 11), (45, 34), (46, 11), (50, 11), (50, 35), (50, 77), (50, 84), (50, 96), (51, 11), (52, 63), (53, 63), (54, 63), (55, 77), (56, 63), (59, 63), (60, 63), (61, 11), (62, 11), (66, 11), (67, 71), (69, 77), (72, 82), (73, 2), (73, 77), (78, 11), (78, 35), (81, 11), (82, 11), (82, 77), (86, 82), (88, 11), (89, 21), (93, 11), (93, 35), (95, 4), (97, 11), (97, 77), (97, 96), (99, 77), (100, 11), (102, 77), (107, 11), (107, 42), (107, 77), (108, 85), (109, 11), (111, 11), (112, 35), (113, 11), (113, 77), (114, 11), (116, 11), (116, 77), (117, 4), (117, 77), (118, 4), (118, 77), (120, 11), (120, 77), (120, 82), (123, 11), (124, 4), (124, 11), (124, 77), ...]

[317129 rows x 0 columns]

In [0]:
# # Pricing Table

import requests
import datetime
import pandas as pd


def daily_price_historical(symbol, comparison_symbol, all_data=True, limit=1, aggregate=1, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df


pricing_list = []

for symbol in list(df_names['symbol']):
# initialise scraper with time interval
    try:
        df = daily_price_historical(symbol, 'USD')
        df['name'] = symbol
        pricing_list.append(df)
    except:
        df2 = pd.DataFrame(columns = df.columns)
        pricing_list.append(df2)


pricing_df = pd.concat(pricing_list)



pricing_df2 = pricing_df.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'name', right_on = 'symbol')
pricing_df3 = pricing_df2.drop(['name', 'symbol', 'time'], axis = 1)


#Make it a datetime object 
pricing_df3['timestamp'] = pd.to_datetime(pricing_df3['timestamp'])


#Rename timestamp to date
pricing_df3.rename(columns = {'timestamp':'date'}, inplace = True)


pricing_df3.set_index(['date', 'coin_id'], inplace = True)

# In[392]:


pricing_df3.head()


,,close,high,low,open,volumefrom,volumeto
date,coin_id,,,,,,
2017-08-11,0,0.06687,0.4444,0.06687,0.06950,39437.24,2994.43
2017-08-12,0,0.16250,0.1830,0.06687,0.06687,6232.81,765.38
2017-08-13,0,0.19000,0.3880,0.12500,0.16250,10382.08,1885.11
2017-08-14,0,0.32000,0.3300,0.13100,0.19000,7936.82,1788.88
2017-08-15,0,1.00000,1.8000,0.32000,0.32000,3467.80,2905.58


In [0]:
# # Google Trends table

!pip install pytrends
!pip install cryptory
import cryptory
from cryptory import Cryptory

# initialise object 
# pull data from start of 2017 to present day
my_cryptory = Cryptory(from_date = "2015-01-01",to_date="2019-06-01", ascending=True)

#Pulling google trends data from cryptory 

i=1
google_data_list = []
for name in list(df_names['name']):
    if(i>0 and i<101):
        i=i+1
        kw_list = []
        kw_list.append(name)
        try:
            data = my_cryptory.get_google_trends(kw_list)
            google_data_list.append(data)
        except:
            continue;


trend_df1 = pd.concat(google_data_list)

trend_df2 = pd.DataFrame(trend_df1.pivot_table(index = 'date').unstack()).reset_index()


#Adding coin_id column
trend_df3 = trend_df2.merge(df_names[['name', 'coin_id']], how = 'left', left_on = 'level_0', right_on = 'name')

#Dropping columns
trend_df3 = trend_df3.drop(['level_0', 'name'], axis = 1)

#Rename columns
trend_df3.columns = ['date', 'trend', 'coin_id']

trend_df3.set_index(['date', 'coin_id'], inplace = True)

trend_df3.head()


  Stored in directory: /root/.cache/pip/wheels/ec/b4/6f/6ae72a8dc1ca028c78d88b90381b34166b2b6d61c99349bf4c
Successfully built cryptory


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,,trend
date,coin_id,
2015-01-01,0,4.145594
2015-01-02,0,20.267349
2015-01-03,0,27.176672
2015-01-04,0,24.873564
2015-01-05,0,26.255429


In [0]:


# # Twitter Table

import re
import csv
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML


import pandas as pd
tweets_df = pd.DataFrame({'Date': [], 'No. of Tweets': [], 'Coin': []})

for i in range(0,len(df_names['symbol'])-1):
    #len(coin_name["symbol"])-1

    coin = df_names['symbol'][i]
    url = 'https://bitinfocharts.com/comparison/tweets-'+coin.lower()+'.html'
    headers = {'User-Agent': "Chrome/54.0.2840.90"}
    response = requests.get(url, headers=headers)
    html = response.text 

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    x = soup.find_all('script')

    data_1 = re.findall(r'(\[new\sDate.*\]])', str(x))
    data_1 = str(data_1)
    
    
    if data_1 == '[]':
        continue
    data_2 = data_1.split("],[")
    data_2[0] = data_2[0][3:]
    data_2[len(data_2) - 1] = data_2[len(data_2) - 1][:-4]
    data_pd = pd.DataFrame(data_2)
    data_clean = data_pd[0].str.split(",", expand = True)
    data_clean = data_clean.iloc[:, 0:2]
    data_clean.columns = ['Date', 'No. of Tweets']
    data_clean["Date"] = data_clean["Date"].str.slice(10,20)
    data_clean["Coin"] = coin.upper()
    tweets_df = tweets_df.append(data_clean, ignore_index = True)


#Convert Date into datetime object
tweets_df['Date'] = tweets_df['Date'].astype('datetime64')


#Remove the null rows
tweets_df = tweets_df[tweets_df['No. of Tweets'] != 'null']


#Convert No. of Tweets into int object
tweets_df['No. of Tweets'] = tweets_df['No. of Tweets'].astype('int')


#Adding coin_id column
tweets_df = tweets_df.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'Coin', right_on = 'symbol')

#Drop redundant name columns
tweets_df = tweets_df.drop(['Coin', 'symbol'], axis = 1)


#Change to tweet_count
tweets_df.rename(columns = {'No. of Tweets':'tweet_count'}, inplace = True)

#Set date & coinid as index
tweets_df.set_index(['Date', 'coin_id'], inplace = True)


tweets_df.head()

,,tweet_count
Date,coin_id,
2018-01-08,0,7
2018-01-09,0,20
2018-01-10,0,21
2018-01-11,0,4
2018-01-12,0,6


In [0]:

# # Reddit Subscribers

#Subreddits based on symbol

empty_df = pd.DataFrame(columns = ['date','total_subscribers','subreddit'])
subreddit_list = []
for name in list(df_names['symbol']):
    try:
        subred = my_cryptory.extract_reddit_metrics(subreddit = name, metric = "total-subscribers", col_label="", sub_col=True)
        if (type(subred) == ValueError):
            subreddit_list.append(empty_df)
        else:
            subreddit_list.append(subred)
    except:
        subreddit_list.append(empty_df)


#Concatenate DF's of symbol-based subreddits
reddit_subscribers = pd.concat(subreddit_list)

#Add the coin_id column
reddit_subscribers2 = reddit_subscribers.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'subreddit', right_on = 'symbol')



#Drop the symbol column
reddit_subscribers3 = reddit_subscribers2.drop(['symbol'], axis = 1)


#Subreddits based on name

empty_df = pd.DataFrame(columns = ['date','total_subscribers','subreddit'])
subreddit_list2 = []
for name in list(df_names['name']):
    try:
        subred = my_cryptory.extract_reddit_metrics(subreddit = name, metric = "total-subscribers", col_label="", sub_col=True)
        if (type(subred) == ValueError):
            subreddit_list2.append(empty_df)
        else:
            subreddit_list2.append(subred)
    except:
        subreddit_list2.append(empty_df)


#Concatenate DF's of name-based subreddits
reddit_subscribers_name = pd.concat(subreddit_list2)


#Add the coin_id column
reddit_subscribers_name2 = reddit_subscribers_name.merge(df_names[['name', 'coin_id']], how = 'left', left_on = 'subreddit', right_on = 'name')


#Drop the name column
reddit_subscribers_name3 = reddit_subscribers_name2.drop('name', axis = 1)


# CONCATENATE BOTH SUBSCRIBERS TABLES INTO ONE

reddit_subscribers_final = pd.concat([reddit_subscribers3, reddit_subscribers_name3])


#Set date & coinid as index
reddit_subscribers_final.set_index(['date', 'coin_id'], inplace = True)

#View table
reddit_subscribers_final.head()

,,total_subscribers,subreddit
date,coin_id,,
2015-01-01,2,882,AION
2015-01-02,2,884,AION
2015-01-03,2,885,AION
2015-01-04,2,887,AION
2015-01-05,2,896,AION


In [0]:
#Mount google drive to save files (temporary solution)

from google.colab import drive
drive.mount('drive')
#After authentication, you can copy your csv file.

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
from datalab.context import Context
import datalab.storage as gcs
Context.default().set_project_id('leafy-stock-240814')



gcs.Bucket('python_output').item('reddit_data.csv').write_to(reddit_data.to_csv(encoding='utf-16'),'text/csv')

gcs.Bucket('python_output').item('coin_reddit_join2.csv').write_to(coin_reddit_join2.to_csv(),'text/csv')

gcs.Bucket('python_output').item('trend_df3.csv').write_to(trend_df3.to_csv(),'text/csv')

gcs.Bucket('python_output').item('pricing_df3.csv').write_to(pricing_df3.to_csv(),'text/csv')

gcs.Bucket('python_output').item('tweets_df.csv').write_to(tweets_df.to_csv(),'text/csv')

gcs.Bucket('python_output').item('reddit_subscribers_final.csv').write_to(reddit_subscribers_final.to_csv(),'text/csv')




